# Hyperparameter Tuning in Deep Learning Using `hyperopt`

- It's the process of systematically searching for the best combination of hyperparameter values for a machine learning model
- The process tests out all the provided permutations and evaluates the outcome of the model. In the end, it selects a subset of the hyperparameters that give the best outcome.

> NOTE: Used Google Colab

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.datasets import load_breast_cancer
from sklearn.neural_network import MLPClassifier # Sk Learn's version of NN


- We'll be using `hyperopt` library to perform the hyperparameter tuning
- It defines a search space for the hyperparameter grid
- Then, it uses the TPE (Tree Parzen Estimators) algorithm to optimize the paramters
- Submodules:
  - `hp` has the functions for the hyperparameter search
  - `fmin` takes the objective functions, seach space, and optimization algorithm. Then, performs minimization of the objective function to get the optimal paramters
  - `tpe` implements a tree-structured estimator of the search space

In [2]:
from hyperopt import hp, fmin, tpe

## 1. Data Import and Prep

In [3]:
load_breast_cancer_data = load_breast_cancer()
load_breast_cancer_data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])

In [4]:
print(load_breast_cancer_data['DESCR'])

.. _breast_cancer_dataset:

Breast cancer wisconsin (diagnostic) dataset
--------------------------------------------

**Data Set Characteristics:**

:Number of Instances: 569

:Number of Attributes: 30 numeric, predictive attributes and the class

:Attribute Information:
    - radius (mean of distances from center to points on the perimeter)
    - texture (standard deviation of gray-scale values)
    - perimeter
    - area
    - smoothness (local variation in radius lengths)
    - compactness (perimeter^2 / area - 1.0)
    - concavity (severity of concave portions of the contour)
    - concave points (number of concave portions of the contour)
    - symmetry
    - fractal dimension ("coastline approximation" - 1)

    The mean, standard error, and "worst" or largest (mean of the three
    worst/largest values) of these features were computed for each image,
    resulting in 30 features.  For instance, field 0 is Mean Radius, field
    10 is Radius SE, field 20 is Worst Radius.

    - 

In [5]:
load_breast_cancer_data['target_names']

array(['malignant', 'benign'], dtype='<U9')

In [6]:
#define X and y
X, y = load_breast_cancer_data['data'], load_breast_cancer_data['target']

In [7]:
#optional - build it as a dataframe
df = pd.DataFrame(X, columns=load_breast_cancer_data['feature_names'])
df['cancer'] = y
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,cancer
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


In [8]:
df['cancer'].value_counts(normalize=True)

,proportion
cancer,
1,0.627417
0,0.372583


In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

## 2. Initialize The Hyperparameter Space

Possible Hyperparameters:
```python
class sklearn.neural_network.MLPClassifier(hidden_layer_sizes=(100,), activation='relu', *, solver='adam', alpha=0.0001, batch_size='auto', learning_rate='constant', learning_rate_init=0.001, power_t=0.5, max_iter=200, shuffle=True, random_state=None, tol=0.0001, verbose=False, warm_start=False, momentum=0.9, nesterovs_momentum=True, early_stopping=False, validation_fraction=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-08, n_iter_no_change=10, max_fun=15000)
```



In [10]:
param_grid = {
    # adjust the number of neurons and hidden layers
    'hidden_layer_sizes': hp.choice('hidden_layer_sizes', [(10), (10, 20), (20,30,40)]), # e.g. last one is 3 hidden layers with 20, 30, 40 neurons respectivly
    # choose the activation function - here it will try every possible combination
    # e.g. for 3 alyers: relu, relu, relu ; tanh, relu, relu; tanh, tanh, relu
    'activation': hp.choice('activation', ['relu', 'tanh']),
    # choose the optmizer algorithm (solver)
    'solver': hp.choice('solver', ['adam', 'sgd']),
    # strength of l2 regularization
    'alpha': hp.choice('alpha', [0.0001, 0.05]),
    # adjust the learning rate method
    'learning_rate': hp.choice('learning_rate', ['constant', 'adaptive']),
    # adjust the learning rate value
    'learning_rate_init': hp.choice('learning_rate_init', [0.001, 0.01]),
    #'max_iter': hp.choice('max_iter', [200, 500]),
}

## 3. Define The Objective Function

In [11]:
# define the objective function to be minimized
def objective_func(input_params):
  '''
  This function takes a set of hyperparamters to train the MLPClassifier and returns the best accuracy score
  '''

  # create a dictionary to pull in the hyperparamters grid
  params_dict = {
      'hidden_layer_sizes': input_params['hidden_layer_sizes'],
      'activation': input_params['activation'],
      'solver': input_params['solver'],
      'alpha': input_params['alpha'],
      'learning_rate': input_params['learning_rate'],
      'learning_rate_init': input_params['learning_rate_init'],
      'random_state': 1000, # set a random seed for reproducibility
      'max_iter': 100 # set the num of iterations
  }

  # instantiate the MLP Classifier model with the given hyperparameters
  mlp_model = MLPClassifier(**params_dict)

  # perform 4-fold cross-validation
  model_score = cross_val_score(mlp_model, X_train, y_train, cv=4, scoring='accuracy').mean()

  # to maximize accuracy we calcualte the negative of the model score
  return -model_score

## 4. Perform Hyperparameter Tuning Using TPE

In [12]:
grid_search = fmin(
    fn=objective_func, # my defined obj function
    space=param_grid, #my custom hyperparamters
    algo=tpe.suggest, # optimization algorithm for hyperparamter tuning
    max_evals=50 #, rstate=np.random.default_rng(123)
)

  2%|▏         | 1/50 [00:01<01:21,  1.66s/trial, best loss: -0.6291218479985893]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



  8%|▊         | 4/50 [00:03<00:34,  1.33trial/s, best loss: -0.9153808852054311]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



 10%|█         | 5/50 [00:04<00:36,  1.24trial/s, best loss: -0.9153808852054311]

...

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



 96%|█████████▌| 48/50 [00:29<00:00,  2.53trial/s, best loss: -0.9319123611356022]

/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



100%|██████████| 50/50 [00:30<00:00,  1.63trial/s, best loss: -0.9319123611356022]


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (100) reached and the optimization hasn't converged yet.
  warnings.warn(



## 5. Get The Best Hyperparameters

In [14]:
grid_search

{'activation': 0,
 'alpha': 0,
 'hidden_layer_sizes': 0,
 'learning_rate': 0,
 'learning_rate_init': 1,
 'solver': 0}

- 'activation': 0, -> **relu**
- 'alpha': 0 -> **0.0001**
- 'hidden_layer_sizes': 0 -> **1 hidden layer with 10 neurons**
- 'learning_rate': 0 -> **'constant'**
- 'learning_rate_init': 1 -> **0.01**
- 'solver': 0 -> **'adam'**

## 6. Test The Best Hyperparameters

In [15]:
best_mlp_model = MLPClassifier(
    hidden_layer_sizes=10,
    activation='relu',
    solver='adam',
    alpha=0.0001,
    learning_rate='constant',
    learning_rate_init=0.01)

best_score = cross_val_score(best_mlp_model, X_train, y_train, cv=4, scoring='accuracy').mean()
print('Best Score with Best Hyperparameters',best_score)

Best Score with Best Hyperparameters 0.9530285663904072


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
